In [1]:
import numpy as np
import scipy as sp
from scipy import optimize,special,integrate
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

import copy

from lanczos_bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [3]:
def exact_lanczos_mv(A,q0,k,B=None,reorth=True):
    """
    run Lanczos with reorthogonalization
    
    Input
    -----
    A : entries of diagonal matrix A
    q0 : starting vector
    k : number of iterations
    B : entries of diagonal weights for orthogonalization
    """
    
    n = A.shape[0]
    
    if B is None:
        B = np.ones(n,dtype=A.dtype)
    
    Q = np.zeros((n,k),dtype=A.dtype)
    a = np.zeros(k,dtype=A.dtype)
    b = np.zeros(k,dtype=A.dtype)
    
    Q[:,0] = q0 / np.sqrt(q0*B@q0)
    
    for i in range(1,k+1):
        # expand Krylov space

      #  if i>1:
       #     print(b[i-2],qi@Q[:,i-2])

        qi = A@Q[:,i-1] - b[i-2]*Q[:,i-2] if i>1 else A@Q[:,i-1]
        
        a[i-1] = (qi*B)@Q[:,i-1]
        qi -= a[i-1]*Q[:,i-1]
        
        if reorth:
            qi -= Q[:,:i-2]@(Q[:,:i-2].T@(B*qi))
            
        b[i-1] = np.sqrt((qi*B)@qi)
        if i < k:
            Q[:,i] = qi / b[i-1]
                
    return Q,(a,b)

In [4]:
dl=.005
lam = np.hstack([np.arange(-50,-1+dl/2,dl),np.arange(1,500+dl/2,dl)])

A,C = (0,1)
a,c = (1,.05)
A_mat = lam

n = len(A_mat)
b_vec = np.ones(n,dtype=np.double)
b_vec /= np.linalg.norm(b_vec)

M = lambda x: (A*x**2+C)
N = lambda x: (a*x**2+c)
f = lambda x: M(x)/N(x)

fAb = f(A_mat)*b_vec

K = 500
Q,(a_,b_) = exact_lanczos(A_mat,b_vec,K,reorth=False)

In [5]:
%%timeit -r 1
K = 500
Q,(a_,b_) = exact_lanczos_mv(sp.sparse.spdiags(A_mat,0,n,n),b_vec,K,reorth=False)

2.75 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%%timeit -r 10
lan_lm= streaming_banded_rational(n,K,(A,0,C),(a,0,c))
for j in range(K):
    lan_lm.read_stream(Q[:,j],a_[j],b_[j])
lan_lm.finish_up() 

1.62 s ± 15.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [7]:
%%timeit -r 10
T = np.diag(a_[:K]) + np.diag(b_[:K-1],1) + np.diag(b_[:K-1],-1)
fTe0 = np.linalg.solve(a*T@T+c*np.eye(K),(A*T@T+C*np.eye(K))[:,0])
lank_FA = np.zeros(n)
for i in range(K):
    lank_FA += Q[:,i]*fTe0[i]

577 ms ± 5.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [8]:
b = 500
A = sp.sparse.spdiags(np.ones((b,n)),list(range(-b//2,b//2)),n,n)

In [9]:
%%timeit -r 1
K = 500
Q,(a_,b_) = exact_lanczos_mv(A,b_vec,K,reorth=False)

22.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
